In [8]:
import spacy
import pandas as pd

In [9]:
df =pd.read_csv('https://raw.githubusercontent.com/codebasics/nlp-tutorials/main/14_word_vectors_spacy_text_classification/Fake_Real_Data.csv')

In [10]:
df['label_num'] = df['label'].map({'Fake' : 0, 'Real': 1})
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
df['vector']= df.Text.apply(lambda x: nlp(x).vector)

In [ ]:
df.head()

In [ ]:
print(df.vector.size)
print(df.label.size)

In [ ]:
print(df.vector[0])
print(df.vector[0].size, df.vector[1].size, df.vector[9899].size)
print(df.vector[0][0])

##Gensim

In [ ]:
import gensim.downloader as api

wv= api.load("word2vec-google-news-300")

In [ ]:
print(wv["food"])
print(wv.similarity(w1="food", w2="eat"))

In [11]:
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import nltk

In [12]:
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [16]:
newsVec= [nltk.word_tokenize(title) for title in df['Text']]


skipgram

In [17]:
model = Word2Vec(newsVec, min_count=1, vector_size=300, sg=1)

In [44]:
model

In [18]:
model.save("word2vec.model")

model = Word2Vec.load("word2vec.model")

In [19]:
vector = model.wv[newsVec[0]]
print(vector.size)

94200


In [20]:
df.head()

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [23]:
df['vector_2']= df['Text'].apply(lambda x: model.wv.get_mean_vector(nltk.word_tokenize(x)))

In [24]:
df.head()

,Text,label,label_num,vector_2
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[0.009888535, 0.049982857, -0.020497894, 0.009..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[0.019001149, 0.057972554, -0.016529966, 0.018..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[0.027877212, 0.04744897, 0.0008081974, 0.0093..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[0.0064036, 0.05128586, -0.017914515, 0.019432..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[0.015653268, 0.052231744, -0.011281588, 0.023..."


In [25]:
df['vector_2'][0].size

300

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(df['vector_2'], df.label_num, test_size=0.2)

x_train[:2]

3925    [0.016969893, 0.044704027, -0.0049398965, 0.01...
4422    [0.02639262, 0.055609934, -0.01359403, 0.02027...
Name: vector_2, dtype: object

In [38]:
import numpy as np

x_train_2d= np.stack(x_train)
x_test_2d= np.stack(x_test)

x_train_2d.shape

(7920, 300)

In [39]:
x_train_2d[:2]

array([[ 0.01696989,  0.04470403, -0.0049399 ,  0.01952771, -0.01483193,
        -0.04015526,  0.04971683,  0.06981058,  0.00509168, -0.02447841,
         0.0062531 , -0.03080145,  0.00503368,  0.00072483, -0.03488218,
        -0.03422032,  0.04249819,  0.02396558,  0.00212652,  0.00021456,
        -0.00229481, -0.01144091,  0.03855655,  0.02536898,  0.03218386,
        -0.01543137, -0.0593467 ,  0.03294964, -0.03162909, -0.05746955,
         0.00660292,  0.00024356,  0.03769996, -0.01538338, -0.00937818,
         0.01541084,  0.04786063, -0.03410917, -0.00878159,  0.00156833,
        -0.03499211, -0.01953335,  0.0153579 , -0.00018182,  0.01973804,
         0.01938431,  0.01293206,  0.01187576,  0.00960587,  0.04174666,
         0.01771951,  0.00116185, -0.00851932, -0.00389221, -0.01432044,
         0.0551915 ,  0.04410065, -0.0121967 ,  0.03091349,  0.00599466,
        -0.00495872, -0.02731344,  0.02868343,  0.01825003, -0.02418791,
         0.07106697, -0.00536268,  0.02085987, -0.0

In [43]:
from sklearn.metrics import classification_report

clf= RandomForestClassifier()
clf.fit(x_train_2d, y_train)

y_pred = clf.predict(x_test_2d)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1006
           1       0.99      1.00      0.99       974

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

